<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/Archivematica%E3%81%AEAPI%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArchivematicaのAPIを使ってみる

## サーバの設定

In [34]:
hostname = "matica.aws.ldas.jp"
endpoint = f"http://{hostname}:81/api"
username = "demo"
api_key = "b9f20c2a99c0e2d91c53061b2834a1b68a5f8b7b"
location_uuid = "a71d85ac-aeb0-4258-a931-688d5a84a87b"

## Transferの設定

In [59]:
name = "mc_api_transfer"
type = "standard"
accession = "2023-1234"
paths = ["files/jpg"]
row_ids = [""]

## base64へエンコード

In [60]:
import base64

paths_encoded = []

for path in paths:
  path_encoded = base64.b64encode(f"{location_uuid}:{path}".encode()).decode()
  paths_encoded.append(path_encoded)

paths_encoded

['YTcxZDg1YWMtYWViMC00MjU4LWE5MzEtNjg4ZDVhODRhODdiOmZpbGVzL2pwZw==']

In [61]:
import requests

data = {
    "name": name,
    "type": type,
    "accession": accession,
    "paths[]": paths_encoded,
    "row_ids[]": row_ids
}

headers = {'Authorization': f'ApiKey {username}:{api_key}'}

response = requests.post(f'{endpoint}/transfer/start_transfer/', headers=headers, data=data)

response

<Response [200]>

In [62]:
response.json()

{'message': 'Copy successful.',
 'path': '/var/archivematica/sharedDirectory/watchedDirectories/activeTransfers/standardTransfer/mc_api_transfer-54d2b498-9071-41fe-bd30-4e767c51422c/'}

# List unapproved transfer(s)


In [63]:
response_2 = requests.get(f'{endpoint}/transfer/unapproved/', headers=headers)
response_2.json()

{'results': [{'type': 'standard',
   'directory': 'mc_api_transfer-0a6699b8-be29-4d6b-90ee-70c471ebfe8f',
   'uuid': '0a6699b8-be29-4d6b-90ee-70c471ebfe8f'},
  {'type': 'standard',
   'directory': 'mc_api_transfer-54d2b498-9071-41fe-bd30-4e767c51422c',
   'uuid': '54d2b498-9071-41fe-bd30-4e767c51422c'}],
 'message': 'Fetched unapproved transfers successfully.'}

# Approve transfer(s)

In [64]:
directory = response.json()["path"].split("/")[-2]
directory

'mc_api_transfer-54d2b498-9071-41fe-bd30-4e767c51422c'

In [65]:
data = {
    "type": type,
    "directory": directory
}

headers = {'Authorization': f'ApiKey {username}:{api_key}'}

response3 = requests.post(f'{endpoint}/transfer/approve/', headers=headers, data=data)

response3.json()

{'message': 'Approval successful.',
 'uuid': '54d2b498-9071-41fe-bd30-4e767c51422c'}

In [66]:
transfer_UUID = response3.json()["uuid"]
transfer_UUID

'54d2b498-9071-41fe-bd30-4e767c51422c'

# status

In [68]:
response5 = requests.get(f'{endpoint}/transfer/status/{transfer_UUID}', headers=headers)

response5.json()

{'type': 'transfer',
 'path': '/var/archivematica/sharedDirectory/currentlyProcessing/mc_api_transfer-54d2b498-9071-41fe-bd30-4e767c51422c/',
 'directory': 'mc_api_transfer-54d2b498-9071-41fe-bd30-4e767c51422c',
 'name': 'mc_api_transfer',
 'uuid': '54d2b498-9071-41fe-bd30-4e767c51422c',
 'microservice': 'Create SIP from transfer objects',
 'status': 'COMPLETE',
 'sip_uuid': '8a1d2e20-0fae-4333-be72-0df8f97e81ce',
 'message': 'Fetched status for 54d2b498-9071-41fe-bd30-4e767c51422c successfully.'}

In [69]:
sip_uuid = response5.json()["sip_uuid"]
sip_uuid

'8a1d2e20-0fae-4333-be72-0df8f97e81ce'

# （参考） completed

In [30]:
response_completed = requests.get(f'{endpoint}/transfer/completed/', headers=headers)
response_completed.json()


{'results': ['0ca902eb-aafd-4f63-a37f-a0959457335c',
  '128ceb66-a681-491b-aede-f596a40eebc4',
  '14229ba8-5687-4912-a896-ff4a64a220fa',
  '14a76af3-ea4e-4cea-be48-34ab6251ecfc',
  '171cadb9-dd39-4b4b-830f-f8505e0dc3c1',
  '247b8a58-0e0a-46f3-a44a-ec6beb1072ce',
  '325c58e7-f178-42ce-bcbb-7013c45dd7f4',
  '4ec77ddf-b21b-4b1d-aa9a-029a34d639e1',
  '4f6a69ec-1b67-45e6-ac68-a597823a044e',
  '7acfe224-f27b-463f-90f6-f3b96a512d03',
  'c93bf30f-b824-4881-adb8-a4bccb7e0046',
  'cdf11076-8b4f-419f-8854-e36759d98d79',
  'e4fdfc7a-6770-458f-95d1-e7a5bad80e32'],
 'message': 'Fetched completed transfers successfully.'}

# ingest

In [73]:
response4 = requests.get(f'{endpoint}/ingest/status/{sip_uuid}', headers=headers)

response4.json()

{'type': 'SIP',
 'path': '/var/archivematica/sharedDirectory/currentlyProcessing/mc_api_transfer-8a1d2e20-0fae-4333-be72-0df8f97e81ce/',
 'directory': 'mc_api_transfer-8a1d2e20-0fae-4333-be72-0df8f97e81ce',
 'name': 'mc_api_transfer',
 'uuid': '8a1d2e20-0fae-4333-be72-0df8f97e81ce',
 'microservice': 'Remove the processing directory',
 'status': 'COMPLETE',
 'message': 'Fetched status for 8a1d2e20-0fae-4333-be72-0df8f97e81ce successfully.'}

# ダウンロード

In [74]:
from requests.auth import HTTPBasicAuth

uuid = response4.json()["uuid"]

username="demo"
password="Nd4Ev3XJ"

endpoint2 = f"http://{hostname}:8001/api"

url = f"{endpoint2}/v2/file/{uuid}?format=json"

# print(url)

response6 = requests.get(url, headers=headers, auth=HTTPBasicAuth(username, password))

response6.json()

{'current_full_path': '/var/archivematica/sharedDirectory/www/AIPsStore/8a1d/2e20/0fae/4333/be72/0df8/f97e/81ce/mc_api_transfer-8a1d2e20-0fae-4333-be72-0df8f97e81ce.7z',
 'current_location': '/api/v2/location/b9661f20-63fa-4843-98aa-05f8c6228c1e/',
 'current_path': '8a1d/2e20/0fae/4333/be72/0df8/f97e/81ce/mc_api_transfer-8a1d2e20-0fae-4333-be72-0df8f97e81ce.7z',
 'encrypted': False,
 'misc_attributes': {},
 'origin_pipeline': '/api/v2/pipeline/e395113a-2885-4163-bf5a-a2786dad948e/',
 'package_type': 'AIP',
 'related_packages': [],
 'replicas': [],
 'replicated_package': None,
 'resource_uri': '/api/v2/file/8a1d2e20-0fae-4333-be72-0df8f97e81ce/',
 'size': 20390998,
 'status': 'UPLOADED',
 'uuid': '8a1d2e20-0fae-4333-be72-0df8f97e81ce'}

In [76]:
url2 = f"{endpoint2}/v2/file/{uuid}/download/"

response7 = requests.get(url2, headers=headers, auth=HTTPBasicAuth(username, password))

content = response7.content

output_path = "test.7z"

with open(output_path ,mode='wb') as f: # wb でバイト型を書き込める
  f.write(content)

http://matica.aws.ldas.jp:8001/api/v2/file/8a1d2e20-0fae-4333-be72-0df8f97e81ce/download/


In [77]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>